In [1]:
# Import necessary libraries
import overpy
import json
import geopandas as gpd
import pandas as pd
from pyproj import Geod
import shapely
import jaydebeapi as jdbc
import jpype
import os
import plotly.express as px
from tqdm import tqdm
from collections import Counter
from shapely import affinity
import numpy as np

In [ ]:
#input is an output from 1preprocessing_area_perimeter
input_df = gpd.read_parquet(r'TN_residential_A_PM_SMOD.parquet')
input_df["longitude"]
#input_df.image_ML_type.value_counts()

In [ ]:

# Convert 'latitude' and 'longitude' to float 
input_df['latitude'] = input_df['latitude'].astype(float)
input_df['longitude'] = input_df['longitude'].astype(float)


print(input_df.head())
print(input_df.columns)

input_df["longitude"]

In [ ]:
# connect to the IBM DB2 function
def connect_to_db():

    jar = "db2jcc4.jar"
    # os.environ['PATH'] = 'C:\Program Files\Java\jdk-22.0.1\\bin:' + os.environ['PATH']
    #way to java needs to be changed accordingly
    os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-17\\bin\server"
    os.environ["CLASSPATH"] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        pass
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    # create JDBC connection
    #['username', 'password'] needs to be provided
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                'jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;',
                ['username', 'password'],
                'db2jcc4.jar')
    
    return conn


def get_building_density(cursor, long, lat):
    '''
        Fetch all OSM buildings from desired DB2 table
    '''
    columns = ['id', 'latitude', 'longitude']
    
    # sql statement for selecting entries by defined rectangle boundaries
    # name of database needs to be changed accordingly
    sql = f"""
        select id, latitude, longitude from USER1.FEATURES_DB_TAMIL_NADU
        WHERE 
            LATITUDE <= {lat}+0.0045121 AND LATITUDE >= {lat}-0.0045121 AND 
            LONGITUDE >= {long}-0.00451369 AND LONGITUDE <= {long}+0.00451369
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)

        return df

    

In [ ]:
output_df = input_df.copy()
output_df.insert(6, "building_density_500", 0)
output_df.insert(6, "building_density_250", 0)
output_df.insert(6, "building_density_100", 0)
output_df.insert(6, "building_density_50", 0)

myconn = connect_to_db()
mycursor = myconn.cursor()
progress = 0
max = output_df.shape[0]

for index, row in output_df.iterrows():
    progress = progress + 1
    print("{0:d}/{1:d} ({2:d}%)".format(progress,max,round(progress*100/max)))
    print(row['longitude'],row['latitude'])
    if progress % 500 == 0:
        print("Reconnecting...")
        mycursor.close()
        myconn.close()
        myconn = connect_to_db()
        mycursor = myconn.cursor()
    neighbors500 = get_building_density(mycursor,row['longitude'],row['latitude'])
    neighbors250 = neighbors500.query("(latitude <= "+str(row['latitude'])+"+0.00225605) and (latitude >= "+str(row['latitude'])+"-0.00225605) and (longitude >= "+str(row['longitude'])+"-0.002256845) and (longitude <= "+str(row['longitude'])+"+0.002256845)")
    neighbors100 = neighbors500.query("(latitude <= "+str(row['latitude'])+"+0.00090242) and (latitude >= "+str(row['latitude'])+"-0.00090242) and (longitude >= "+str(row['longitude'])+"-0.000902738) and (longitude <= "+str(row['longitude'])+"+0.000902738)")
    neighbors50 = neighbors500.query("(latitude <= "+str(row['latitude'])+"+0.00045121) and (latitude >= "+str(row['latitude'])+"-0.00045121) and (longitude >= "+str(row['longitude'])+"-0.000451369) and (longitude <= "+str(row['longitude'])+"+0.000451369)")
    present = neighbors500.query("(latitude == "+str(row['latitude'])+") and (longitude == "+str(row['longitude'])+")")
    building_present = 0
    if present.shape[0] == 0:
        building_present = 1
        #print("Building not from VIDA")
    output_df.at[index, "building_density_500"] = neighbors500.shape[0]+building_present
    output_df.at[index, "building_density_250"] = neighbors250.shape[0]+building_present
    output_df.at[index, "building_density_100"] = neighbors100.shape[0]+building_present
    output_df.at[index, "building_density_50"] = neighbors50.shape[0]+building_present

mycursor.close()
myconn.close()

output_df

In [ ]:
print(output_df['building_density_500'].describe())
print(output_df['building_density_250'].describe())
print(output_df['building_density_100'].describe())
print(output_df['building_density_50'].describe())

In [ ]:
main_df = output_df
print(len(main_df))
main_df["building_density_100"].value_counts()
main_df = main_df[main_df["building_density_100"] != 1]
print(len(main_df))
main_df.to_parquet(r"TN_residential_A_PM_BD.parquet")

In [ ]:
# Creating plot
main_df.boxplot(column="building_density_50", by="L1_5_class")


In [ ]:
# Creating plot
main_df.boxplot(column="building_density_100", by="L1_5_class")


In [ ]:
# Creating plot
output_df.boxplot(column="building_density_250", by="L1_5_class")


In [ ]:
# Creating plot
output_df.boxplot(column="building_density_500", by="L1_5_class")


In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

def get_inner_perimeter(interiors):
    geod = Geod(ellps="WGS84")
    total = 0
    for interior in interiors:
        total += abs(geod.geometry_area_perimeter(interior)[1])
    return total

output_df.insert(0,"perimeter_in_meters",0)
output_df['perimeter_in_meters'] = output_df["geometry"].apply(lambda g: (abs(geod.geometry_area_perimeter(g)[1]) + get_inner_perimeter(g.interiors)) if (g.geom_type == "Polygon") else (abs(geod.geometry_area_perimeter(g)[1])))

